# Sleep stages classification pipeline
___

This notebooks aims to construct our feature matrix from our sleep records [sleep-edf](https://physionet.org/content/sleep-edfx/1.0.0/) from _physionet_. 

We will reuse the chosen features from our exploration notebook.

Since all of the dataset cannot be loaded in memory at the same time, we will have to implement a pipeline, where each step can then be run with only one recording at a time. At the end of this notebook, we will be able to concatenate all resulting features in a single matrix.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import mne
from mne.datasets.sleep_physionet.age import fetch_data

In [ ]:
SUBJECTS = range(1)
NIGHT_RECORDINGS = [1]
NB_EPOCHS_AWAKE_MORNING = 50

SAMPLING_FREQ = 100
EPOCH_DURATION = 30. # in seconds
MAX_TIME = EPOCH_DURATION - 1. / SAMPLING_FREQ  # tmax in included

EEG_CHANNELS = [
    'EEG Fpz-Cz',
    'EEG Pz-Oz'
]

# Values
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4

ANNOTATIONS_EVENT_ID = {
    'Sleep stage W': W,
    'Sleep stage 1': N1,
    'Sleep stage 2': N2,
    'Sleep stage 3': N3,
    'Sleep stage 4': N3,
    'Sleep stage R': REM
}

EVENT_ID = {
    "W": W,
    "N1": N1,
    "N2": N2,
    "N3": N3,
    "REM": REM
}

## Preprocessing
___

Our dataset consists of 39 recordings, each containing about 20 hours of EEG, EOG, EMG, and other signals, sampled at 100 or 1Hz.

### 0. Loading recordings informations
___

This file contains information about when a recording was started, at which time the subject went to bed and the amount of sleep he got.

In [ ]:
df_records = pd.read_csv("data/recordings-info.csv")
df_records.head(5)




### 1. Retrieve data from file and convert to dataframe
___

This step includes:
- Retrieving edf file
- Excluding none EEG channels
- Crops the nights recording between the moment the lights are turned off and the subject wakes up.

In [ ]:
def fetch_signal(psg_file_name, hypno_file_name):
    """
    returns: mne.Raw of the whole night recording
    """
    raw_data = mne.io.read_raw_edf(psg_file_name, preload=True, stim_channel=None, verbose=False)
    annot = mne.read_annotations(hypno_file_name)
    raw_data.set_annotations(annot, emit_warning=False)
    
    return raw_data

def drop_other_channels(raw_data, channel_to_keep):
    """
    returns: mne.Raw with the two EEG channels and the signal
        between the time the subject closed the lights and the time
        at which the subject woke up
    """
    raw_data.drop_channels([ch for ch in raw_data.info['ch_names'] if ch != channel_to_keep])
    
    return raw_data

def convert_to_epochs(raw_data, info):
    """
    returns: mne.Epochs, where the epochs are only choosen if the subject was in bed.
    """

    # Number of seconds since file began
    closed_lights_time = info['LightsOffSecond']
    woke_up_time = closed_lights_time + info['NightDuration'] + NB_EPOCHS_AWAKE_MORNING*EPOCH_DURATION
    
    events, _ = mne.events_from_annotations(
        raw_data,
        event_id=ANNOTATIONS_EVENT_ID,
        chunk_duration=EPOCH_DURATION,
        verbose=False)
    
    raw_data.crop(tmin=closed_lights_time, tmax=woke_up_time)
    
    return mne.Epochs(
        raw=raw_data,
        events=events,
        event_id=EVENT_ID,
        tmin=0.,
        tmax=MAX_TIME,
        preload=True,
        baseline=None,
        verbose=False)


def convert_to_matrices(data):
    """
    data: mne.Epochs with only one EEG channel
    
    returns
        - X: Matrix of input values, of size (nb_epochs, sampling_rate*epoch_length=3000)
        - y: Vector of observation labels, of size (nb_epochs,)
    """
    df = data.to_data_frame(picks="eeg", long_format=True)
    df = df.drop(columns=['ch_type', 'channel'])
    df = df.sort_values(by=['epoch', 'time'])
    
    y = df[['epoch', 'condition']].drop_duplicates(keep="first")['condition'].to_numpy()
    X = np.matrix(
        [df[df['epoch'] == epoch]['observation'].to_numpy() for epoch in df['epoch'].unique()]
    )

    return X,y

## Pipeline
___

In [ ]:
%%time

subject_file_names = fetch_data(subjects=SUBJECTS, recording=NIGHT_RECORDINGS)
psg_file_names = [names[0] for names in subject_file_names]
stage_file_names = [names[1] for names in subject_file_names]

for i in range(len(subject_file_names)):
    raw_data = fetch_signal(psg_file_names[i], stage_file_names[i])

    for channel in EEG_CHANNELS:
        data = raw_data.copy()
        data = drop_other_channels(data, channel)
        data = convert_to_epochs(data, df_records.iloc[i])
        print(data)

        X, y = convert_to_matrices(data)
        print(X, y)
        del data, X, y

    del raw_data
